<a id='top'></a>
# Working with NSIDC DAAC/NOAA data in R
---
This notebook was built to provide basic use cases for accessing and working with NSIDC/NOAA data in R. The Jupyter Notebook uses an R kernel so that the R syntax and packages/libraries can be used. For the most part, all code can be copied and pasted directly into R Studio for running in a local environment.

Please send any questions, comments, or use cases of interest to NSIDC User Services at nsidc@nsidc.org

### Notebook Sections:
_Click a link to jump to a section of interest_
1. [Set up](#start) your JupyteR environment
2. [Importing data](#import) from the web (HTTPS and FTP)
3. Working with [NetCDF 4 (.nc)](#netcdf)
4. Working with [HDF5 (.h5)](#hdf)
5. Working with [HDF (.he5)](#he5)
6. Working with [Binary (.bin)](#bin)
7. Working with [multiple datasets](#mult)

---
<a id='start'></a>
## 1. **_Start Here_:** Steps for setting up your local JupyteR environment<h5>(Binder users can skip this step and move to the "Load libraries" cell)</h5>
<b>Step 1.</b> Activate or create a conda environment with all of the packages required for this notebook (_note_: If you only need to run a specific section of the notebook, packages used are listed at the top of each section):<br />

<i>conda create -n chooseyourenvname r-base r-essentials r-ncdf4 r-raster r-gdalUtils r-rgdal r-RCurl r-httr r-getPass r-sf r-rnaturalearth r-rnaturalearthdata</i>

<b>Step 2.</b> Run <i>activate yourenvname</i>

<b>Step 3.</b> Run <i>conda install -c conda-forge jupyterlab</i>

<b>Step 4.</b> Run _jupyter lab_ to launch the notebook interface

   <b><i>Notes:</i></b>
   - Any R package you want to install with _conda_ should be written as "r-<i>packagename</i>"
   - _r-essentials_ includes tidyverse, shiny, dplyr, and other useful packages so you don't need to install them individually.
   - _rgdal_ can sometimes cause trouble for new users installing the package. If you do experience issues, learn more here: https://hdfeos.org/software/gdal.php

---
<b>Load libraries</b>

In [ ]:
libraries <- c("ncdf4", "rgdal", "gdalUtils", "raster", "RCurl", "R.utils", "httr", "getPass", "rnaturalearth", "rnaturalearthdata", "ggplot2", "sf", "sys")
invisible(lapply(libraries, library, character.only = TRUE))

<b>Note:</b> Each section starts with a cell setting a different working directory. You can skip these if you are working from a single directory.

---
<a id='import'></a>
## 2. **Importing NSIDC Datasets** from HTTPS and FTP Sources
Package(s): **RCurl** & **httr** </b>Datasets: [**IceBridge DMS L3 Photogrammetric DEM, Version 1 (IODMS3)**](https://nsidc.org/data/IODMS3/versions/1), [**IMS Daily Northern Hemisphere Snow and Ice Analysis at 1 km, Version 1 (G02156)**](https://nsidc.org/data/G02156/versions/1)

LP DAAC also has an R data access notebook: https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_r/browse/DAACDataDownload.R

#### Set working directory and NASA Earthdata Authentication

In [ ]:
# Check current working directory (usually the directory where Jupyter Lab was launched from)
getwd()

In [ ]:
# Set directory to download files to
usr <- file.path(Sys.getenv("./data"))                           # Retrieve home dir (for netrc file)
if (usr == "") {usr = Sys.getenv("./")}                      # If no user profile exists, use home
netrc <- file.path(usr,'.netrc', fsep = .Platform$file.sep)  # Path to netrc file

[Create](https://nsidc.org/support/how/v0-programmatic-data-access-guide) a <b>.netrc</b> file that contains your NASA Earthdata login credientials manually and place it in your working directory.

In [39]:
# This system for creating a .netrc file currently doesn't work in Jupyter Notebook and will need to be replaced. You will need to manually create the file in the link above.

# # Set a path to .netrc file
# #netrc_path <- "./.netrc"

# # If you don't already have a .netrc file created with your NASA Earthdata Credentials, you will be prompted here to create one.
# if (file.exists(netrc) == FALSE || grepl("urs.earthdata.nasa.gov", readLines(netrc)) == FALSE) {
#   netrc_conn <- file(netrc)

#   # User will be prompted for NASA Earthdata Login Username and Password below
#   writeLines(c("machine urs.earthdata.nasa.gov",
#                sprintf("login %s", getPass(msg = "Enter NASA Earthdata Username:")),
#                sprintf("password %s", getPass(msg = "Enter NASA Earthdata Login Password:"))), netrc_conn)
#   close(netrc_conn)
#   }

#### <u>HTTPS Access</u>

In [ ]:
# Single file:
files <- "https://n5eil01u.ecs.nsidc.org/ICEBRIDGE/IODMS3.001/2013.11.26/IODMS3_20131126_20102450_00025_DEM.tif"

# A list of files:
#files <- c("https://n5eil01u.ecs.nsidc.org/filepath/file.hdf",
#           "https://n5eil01u.ecs.nsidc.org/filepath/file.hdf")

# Text file with list of links:
#files <- readLines("C:/filepath/URL_file_list.txt", warn = FALSE)

# Loop through all files
for (i in 1:length(files)) {
  filename <-  tail(strsplit(files[i], '/')[[1]], n = 1) # Keep original filename

  # Write file to disk (authenticating with netrc) using the current directory/filename
  response <- GET(files[i], write_disk(filename, overwrite = TRUE), progress(),
                  config(netrc = TRUE, netrc_file = netrc_path), set_cookies("LC" = "cookies"))

  # Check to see if file downloaded correctly
  if (response$status_code == 200) {
    print(sprintf("%s downloaded at %s", filename, path))
  } else {
  print(sprintf("%s not downloaded. Verify that your username and password are correct in %s", filename, netrc))
  }
}

#### <u>NSIDC FTP Access</u>
1. <b>Host Name:</b> sidads.colorado.edu <br />
2. <b>User Name:</b> anonymous <br />
3. <b>Password:</b> your e-mail address <br />
4. <b>Directory:</b> /pub/DATASETS/XXXX <br />

#### Getting Connected:

In [ ]:
# This combination will be used throughout this section and shouldn't change
usrpwd <- readline(prompt="Enter anonymous:youremailaddress ")

In [ ]:
# Your URL may change depending on what dataset you're interested in looking at. This will change in this section.
url <- "ftp://sidads.colorado.edu/pub/DATASETS/"

In [ ]:
filenames <- getURL(url, userpwd = usrpwd,
             ftp.use.epsv = FALSE, dirlistonly = TRUE)

In [ ]:
# Uncomment the line below to get a list of sub-directories
cat(filenames)

In [ ]:
# The data will be saved into your current working directory
ims_url = "ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02156/GIS/1km/2021/"
download.file(ims_url, destfile = "ims2021001_1km_GIS_v1.3.tif.gz")

In [ ]:
gunzip("ims2021001_1km_GIS_v1.3.tif.gz", remove=FALSE)

Now the unzipped file should be in your working directory and ready to use!

---
<a id='netcdf'></a>
## 3. Working with <b>NetCDF 4</b> files in R
Package:<b> ncdf4 </b>Dataset:<b> SnowEx 2020 Airborne DEM Mosaic (pre-release)</b>

In [ ]:
# Set data path and filename
path <- "./data"
ncname <- "/SNOWEX2020_EO_PLANE_2020Feb08_mosaicked_APLUW"
ncfname <- paste(path, ncname, ".nc", sep="")

In [ ]:
# open the netCDF file
ncin <- nc_open(ncfname)
print(ncin)

# Open the file using rgdal method 
snex <- "SNOWEX2020_EO_PLANE_2020Feb08_mosaicked_APLUW.nc"

In [ ]:
# Get subdatasets so that you can grab the parameter slice you're interested in
sds <- get_subdatasets(snex)
sds

In [ ]:
# Convert your parameter slice to .tif and save to local directory
gdal_translate(sds[2], dst_dataset = "SNEX20_DEM_NK_QC_02102021_v3.tif")

In [ ]:
# Pull the new .tif raster back into R and examine the data
dem <- raster("SNEX20_DEM_NK_QC_02102021_v3.tif")
show(dem)

In [ ]:
# Set up the projection for the data using WTK
wkt_32613 <- 'PROJCS["WGS 84 / UTM zone 13N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.01745329251994328,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-105],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    AUTHORITY["EPSG","32613"],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH]]'

In [ ]:
crs(dem) <- CRS(SRS_string=wkt_32613)
extent(dem) <- c(-108.0000, 0.0000, -102.0000, 84.0000)
show(dem)
plot(dem)

In [ ]:
# Uncomment the line below to write the raster to your local directory
#writeRaster(dem, "SNEX20_DEM_EPSG32613_NK_QC_02102021.tif", overwrite = TRUE)


[**Back to top**](#top)

---
<a id='hdf'></a>
## 4. Working with <b> HDF5 </b> data in R
Package:<b> rgdal </b>Dataset: **[ICESat-2 L3B Monthly Gridded Atmosphere V003 (ATL16)](https://nsidc.org/data/ATL16)**

In [ ]:
# Convert a single file
path <- "./data"
is2_file <- 'ATL16_20190201001042_05280201_955a2_01.h5'
filepath <- file.path(path, is2_file)
print(filepath)

In [ ]:
# Be careful using this command as the list can be very long with unsubset HDF files.
is2_sds <- get_subdatasets(filepath)
list(is2_sds)

In [ ]:
od <- gdal_translate(is2_sds[9], dst_dataset = "ATL16_global_od_WGS84_PanoplyFix.tif")

In [ ]:
gaf <- raster("ATL16_global_od_WGS84_PanoplyFix.tif")
gaf

In [ ]:
wkt_wgs84 <- 'GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]'

In [ ]:
coastlines <- ne_coastline(scale = 110, returnclass = c("sp", "sf"))
show(coastlines)

In [ ]:
gaf.ext <- raster("ATL16_global_od_WGS84_PanoplyFix.tif")
crs(gaf.ext) <- CRS("+proj=longlat +datum=WGS84 +no_defs")
gaf.ext <- setExtent(gaf.ext, coastlines, keepres=FALSE)
show(gaf.ext)
#gaf.ext <- setExtent(gaf.ext, c(-180,180,-90,90), keepres = FALSE)

In [ ]:
# As of 2/17/2021, ATL16 and ATL17 data needs to be flipped 180 degrees to properly plot the data
gaf.ext.flip <- flip(gaf.ext, direction='y')

In [ ]:
#jpeg(file="ATL16_Global_OD_Flip.jpeg")

plot(gaf.ext, 
      breaks = c(0.0,0.2, 0.4, 0.6, 0.8, 1.0, 3.0), 
      col = rainbow(5),
      main="Global Column Optical Depth",
      legend=FALSE,
      axes=TRUE)
plot(coastlines, add=TRUE)

#dev.off()

In [ ]:
plot(gaf.ext.flip, 
      breaks = c(0.0,0.2, 0.4, 0.6, 0.8, 1.0), 
      col = rainbow(5),
      legend=TRUE,
      axes=TRUE)
plot(coastlines, add=TRUE)

In [ ]:
# Save the plot
# jpeg(file="ATL16_Global_OD_upsidedown.jpeg")
# plot(gaf.ext, 
#      breaks = c(0.2, 0.4, 0.6, 0.8, 1.0), 
#      col = rainbow(5),
#      main=file,
#      legend=FALSE,
#      axes=TRUE)
# plot(coastlines, add=TRUE)
# dev.off()

In [ ]:
# Uncomment the following line if you want to save the final GeoTiff
#writeRaster(gaf.ext.flip, 'ATL16_global_column_od_flip.tif', options=c('TFW=YES', 'XML=YES'))

[**Back to top**](#top)

---
<a id='he5'></a>
## 5. Working with <b>.he5 </b>data in R
Package:<b> rgdal </b>Dataset: **[AMSR-E/AMSR2 Unified L3 Daily 12.5 km Brightness Temperatures, Sea Ice Concentration, Motion & Snow Depth Polar Grids](https://nsidc.org/data/AU_SI12/versions/1)**

In [ ]:
# Convert a single file
amsr_file <- "./data/AMSR_U2_L3_SeaIce12km_B04_20210219.he5"
print(amsr_file)

In [ ]:
# View subdatasets within the HDF file
amsr_sds <- get_subdatasets(amsr_file)
#amsr_sds

In [ ]:
# convert your data slice to .tif and save new file to correct CRS and extent
gdal_translate(amsr_sds[59], dst_dataset = "SI_12km_SH_ICECON_DAY_20210219.tif")
amsr_gaf <- raster("SI_12km_SH_ICECON_DAY_20210219.tif")
show(amsr_gaf)

In [ ]:
wkt_south <- 'PROJCS["WGS 84 / NSIDC Sea Ice Polar Stereographic South",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Polar_Stereographic"],
    PARAMETER["latitude_of_origin",-70],
    PARAMETER["central_meridian",0],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["X",EAST],
    AXIS["Y",NORTH],
    AUTHORITY["EPSG","3976"]]'

In [ ]:
crs(amsr_gaf) <- CRS(SRS_string=wkt_south)
extent(amsr_gaf) <- c(-3950000, 3950000, -3950000, 4350000)

In [ ]:
show(amsr_gaf)
plot(amsr_gaf)

In [ ]:
amsr_gaf_df <- as.data.frame(amsr_gaf, xy=TRUE)
str(amsr_gaf_df)

min <- minValue(amsr_gaf)
max <- maxValue(amsr_gaf)

In [ ]:
ggplot() +
    geom_raster(data = amsr_gaf_df , aes(x = x, y = y, fill = SI_12km_SH_ICECON_DAY_20210219)) +
    scale_fill_viridis_c() +
    coord_quickmap()

In [ ]:
#writeRaster(amsr_gaf, "SI_12km_SH_ICECON_DAY_020821_extent.tif", overwrite = TRUE)

[**Back to top**](#top)

---
<a id='bin'></a>
## 6. Working with <b>Binary (.bin) </b>data in R
Package:<b> rgdal </b>Dataset: **[Sea Ice Concentrations from Nimbus-7 SMMR and DMSP SSM/I-SSMIS Passive Microwave Data, Version 1 (NSIDC-0051)](https://nsidc.org/data/NSIDC-0051/versions/1)**

**Note:** Before you can convert Binary data to a raster, you must first create a _header file (.hdr)_ and place it in your working directory. NSIDC has a _How To_ article that includes the necessary header file information for the Northern and Southern Hemisphere versions of this example data product: https://nsidc.org/support/how/how-do-i-convert-nsidc-0051-sea-ice-concentration-data-binary-geotiff

In [ ]:
nt_file <- "./data/nt_20201031_f17_v1.1_s.bin"
nt_file

In [ ]:
gdal_translate(nt_file, 
               of = "GTiff",
               dst_dataset = "nt_test.tif", 
               a_srs = '+proj=stere +lat_0=-90 +lat_ts=-70 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=6378273 +b=6356889.449 +units=m +no_defs',
               projwin_srs = '+proj=stere +lat_0=-90 +lat_ts=-70 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=6378273 +b=6356889.449 +units=m +no_defs',
               a_nodata = 255, 
               a_ullr = c(-3950000.0, 4350000.0, 3950000.0, -3950000.0),
               scale = c(0, 255),
               output_Raster = TRUE, verbose = FALSE);

In [ ]:
nt_raster <- raster("t_20201031_f17_v1.1_s.tif")
plot(nt_raster)

[**Back to top**](#top)

---
<a id='mult'></a>
## 7. Working with **Multiple Datasets** in R
Package(s): **rgdal, ggplot2** </b>Dataset: **[MEaSUREs inSAR-Based Antarctic Ice Velocity Map, Version 2 (NSIDC-0484)](https://nsidc.org/data/NSIDC-0484/versions/2), [GLIMS Glacier Database](http://glims.colorado.edu/glacierdata/)**

In [ ]:
path <- "./data"
file_path <- file.path(path, "glims_polygons.shp")
file_path
pine_island <- st_read(file_path)

In [ ]:
ggplot() + 
  geom_sf(data = pine_island, size = 1, color = "black", fill = "cyan1") + 
  ggtitle("Pine Island Glacier, Antarctica") + 
  coord_sf()

In [ ]:
meas_file <- "antarctica_ice_velocity_450m_v2.nc"
meas_path <- file.path(path, meas_file)
meas_path

In [ ]:
# View subdatasets within the HDF file
meas_sds <- get_subdatasets(meas_path)
meas_sds

In [ ]:
gdal_translate(meas_sds[3], dst_dataset = "VX.tif") #, a_srs = "EPSG:3412", s_srs = "EPSG:3412") #, projwin_srs = "EPSG:3976")

In [ ]:
vx <- raster("VX.tif")
show(vx)

In [ ]:
pine_island_prj <- st_transform(pine_island, crs = st_crs(vx))

In [ ]:
plot(vx)
plot(pine_island_prj, add=TRUE)

In [ ]:
vx_crop <- crop(vx, pine_island_prj)

In [ ]:
pine_island_prj_df <- fortify(pine_island_prj)
vx_crop_df <- as.data.frame(vx_crop, xy = TRUE) 
str(vx_crop_df)

In [ ]:
ggplot() +
  geom_raster(data = vx_crop_df, 
              aes(x = x, y = y, 
                  fill = VX)) + 
  scale_fill_viridis_c() +  
  scale_alpha(range = c(0.15, 0.65), guide = "none") +  
  ggtitle("Pine Island Ice Velocity") +
  coord_quickmap()

---
[**Back to the top**](#top)